### Analysis of Jafaar's scratcher wins in ~2017
549 wins of $1000.

Find minimum spend across all scratcher games.

In [1]:
import csv
import math
import numpy as np
import scipy as sp
from scipy.stats import binom
from scipy.special import betainc
from scipy.optimize import minimize

def binTail(p, n, t): # upper tail probability for a vector of Binomial(n,p) random variables
    return betainc(n, t - n + 1, p)

import pandas as pd
fn = './jafaar-scratchers.csv'  # games and odds

wins = 549
CHANCE_THRESHOLD =  10**(-7) # one in ten million threshold
MAX_PLAYERS = 6859819  # MAX_PLAYERS of MA in 2017 per census
eps = CHANCE_THRESHOLD/MAX_PLAYERS
TOT_REVENUE = 3517783 # Scratcher revenue in MA, 2017

In [2]:
odds = pd.read_csv(fn)
odds['per_dollar'] = odds['Consolidated odds to win $1,000']/odds['$ cost to play']
oddsBy = odds.groupby('$ cost to play')

In [3]:
# find max for each ticket cost

print(oddsBy['Consolidated odds to win $1,000'].aggregate('max'))
print(oddsBy['Consolidated odds to win $1,000'].aggregate(np.argmax))
inx = oddsBy['Consolidated odds to win $1,000'].aggregate(np.argmax).values
print(inx)

$ cost to play
1     0.000030
2     0.000069
5     0.000340
10    0.000479
Name: Consolidated odds to win $1,000, dtype: float64
$ cost to play
1      4.0
2     20.0
5     53.0
10    76.0
Name: Consolidated odds to win $1,000, dtype: float64
[  4.  20.  53.  76.]


/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [4]:
odds

,Instant ticket game,"Consolidated odds to win $1,000",$ cost to play,per_dollar
0,"$10,000 Holiday Bonus 2015",0.000010,1,0.000010
1,"$10,000 Holiday Bonus 2017",0.000014,1,0.000014
2,10X The Cash 2016,0.000010,1,0.000010
3,9's in a Line 2017,0.000010,1,0.000010
4,$500 a Week For Life 2014,0.000030,1,0.000030
5,Double 8's 2016,0.000010,1,0.000010
6,Double Payday 2018,0.000010,1,0.000010
7,Make My Day™ 2017,0.000010,1,0.000010
8,Wild Doubler 2015,0.000010,1,0.000010
9,"$50,000 Bingo Boxes",0.000028,2,0.000014


In [5]:
game = odds.iloc[inx]
game

,Instant ticket game,"Consolidated odds to win $1,000",$ cost to play,per_dollar
4,$500 a Week For Life 2014,0.000030,1,0.000030
20,Bingo Star Tripler 2013,0.000069,2,0.000035
53,Creepy Cash 2017,0.000340,5,0.000068
76,100X 2017,0.000479,10,0.000048


In [6]:
def findn(p, k, eps, maxiter = 10**4):
    """
    Find the sample size n for a binomial such that the chance of k or more successes in
    n Bernoulli(p) trials is at least eps
    """
    nmax = math.ceil(k/p)
    assert binTail(p, k, nmax) > eps
    while binTail(p, k, nmax) > eps:
        nmax = nmax/2
    nmax = nmax/2
    nmin = k
    assert binTail(p, k, nmin) < eps
    while binTail(p, k, nmin) < eps:
        nmin = 2*nmin
    nmin = 2*nmin
    f = lambda x: binTail(p, k, x)-eps
    return math.ceil(sp.optimize.brentq(f, nmin, nmax, maxiter=maxiter))

In [7]:
minspend = 10**9
minmult = 10**10

for i in range(len(game['Instant ticket game'])):
    r = game.iloc[i]
    p = r['Consolidated odds to win $1,000']
    n = findn(p,wins,eps)
    c = r['$ cost to play']*n
    if c < minspend:
        minspend = c 
        minmult = MAX_PLAYERS*c/TOT_REVENUE    
    print('\n\n', r)
    print(p, wins, n, c, MAX_PLAYERS*c/TOT_REVENUE)
    
print('\n\n', minspend, minmult, minmult*TOT_REVENUE)



 Instant ticket game                $500 a Week For Life 2014
Consolidated odds to win $1,000                  2.97619e-05
$ cost to play                                             1
per_dollar                                       2.97619e-05
Name: 4, dtype: object
2.97619e-05 549 13081486 13081486 25509426.3094


 Instant ticket game                Bingo Star Tripler 2013
Consolidated odds to win $1,000                6.94444e-05
$ cost to play                                           2
per_dollar                                     3.47222e-05
Name: 20, dtype: object
6.94444e-05 549 5606399 11212798 21865409.1977


 Instant ticket game                Creepy Cash 2017
Consolidated odds to win $1,000         0.000340368
$ cost to play                                    5
per_dollar                              6.80736e-05
Name: 53, dtype: object
0.000340368 549 1143923 5719615 11153480.3738


 Instant ticket game                  100X 2017
Consolidated odds to win $1,000    0.0004

In [8]:
f = lambda x: binTail(p, k, x)-eps
k = wins
p = 0.000340368
print(f(5719615/5), binTail(p, wins, 5719615/5), eps)

1.72820942153e-18 1.45793723134e-14 1.457764410402082e-14


In [11]:
TOT_REVENUE/MAX_PLAYERS # sanity check

0.5128098860917467

In [12]:
TOT_REVENUE*11*10**6/(3.9*10**13) # double check

0.9921952051282051